In [21]:
import os
import re
import gensim
import pymorphy2

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format("ruwikiruscorpora_0_300_20.bin.gz", binary=True)
model.init_sims(replace=True)

In [22]:
morph = pymorphy2.MorphAnalyzer()
punct = re.compile('^(.*?)([а-яА-ЯёЁ-]+)(.*?)$')
capit = re.compile('^[А-Я]+$')

In [16]:
pth_source = 'books_before/'
lst = os.listdir(pth_soursce)

In [34]:
cotags = {'ADJF':'ADJ', # pymorphy2: word2vec 
'ADJS' : 'ADJ', 
'ADVB' : 'ADV', 
'COMP' : 'ADV', 
'GRND' : 'VERB', 
'INFN' : 'VERB', 
'NOUN' : 'NOUN', 
'PRED' : 'ADV', 
'PRTF' : 'ADJ', 
'PRTS' : 'VERB', 
'VERB' : 'VERB'}

#### todo

* capitalize
* pos detection
* cashing of word2vec queries
* 1st form extraction from pymorphy2 parse of word2vec response
* pos matching for most similar words
* names detection and excluding from the process

In [ ]:
def find_word(word):
    pass

In [33]:
model.most_similar(['холодный_ADJ'])

[('теплый_ADJ', 0.6912842392921448),
 ('холод_NOUN', 0.6222048997879028),
 ('прохладный_ADJ', 0.6178681254386902),
 ('влажный_ADJ', 0.6009092926979065),
 ('жаркий_ADJ', 0.5878441333770752),
 ('горячий_ADJ', 0.5811808109283447),
 ('ледяной_ADJ', 0.5738131999969482),
 ('сухой_ADJ', 0.5681436657905579),
 ('сырой_ADJ', 0.5447033643722534),
 ('холодно_ADV', 0.5389620661735535)]

In [31]:
for fl in lst:
    print (fl)
    i = 0
    f = open(pth_source + fl, 'r', encoding='utf-8')
    for line in f:
        new_line = []
        i += 1
        if i > 5:
            break
        line = line.strip()
        words = line.split(' ')
        for word in words:
            struct = punct.findall(word)
            if struct:
                struct = struct[0]
            else:
                new_line.append(word)
                continue
            print (struct)
            wordform = struct[1]
            if wordform:
                if capit.search(wordform):
                    new_line.append(word)
                    continue
                parse_result = morph.parse(wordform)[0]
                if 'NOUN' in parse_result.tag or 'VERB' in parse_result.tag or 'INFN' in parse_result.tag or 'ADJF' in in parse_result.tag:
                    lex = parse_result.normal_form
                    tags = parse_result.tag
                    lex_neighb = search_neighbour(lex)
                    if parse_result.tag.POS == 'NOUN' or parse_result.tag.POS == 'ADJF':
                        if parse_result.tag.case == 'nomn' and parse_result.tag.number == 'sing':
                            
                        else:
                            
                    elif parse_result.tag.POS == 'INFN':
                        
                        
                            
                else:
                    new_line.append(word)
                print(parse_result)
            else:
                new_line.append(''.join(struct))
            
            
            #print(morph.parse(line))
        

FathersAndSons.txt
('', 'ОТЦЫ', '')
('', 'И', '')
('', 'ДЕТИ', '')
('', 'Посвящается', '')
[Parse(word='посвящается', tag=OpencorporaTag('VERB,impf,intr sing,3per,pres,indc'), normal_form='посвящаться', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'посвящается', 224, 5),))]
('', 'памяти', '')
[Parse(word='памяти', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='память', score=0.4, methods_stack=((<DictionaryAnalyzer>, 'памяти', 13, 1),)), Parse(word='памяти', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='память', score=0.4, methods_stack=((<DictionaryAnalyzer>, 'памяти', 13, 5),)), Parse(word='памяти', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='память', score=0.1, methods_stack=((<DictionaryAnalyzer>, 'памяти', 13, 2),)), Parse(word='памяти', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='память', score=0.05, methods_stack=((<DictionaryAnalyzer>, 'памяти', 13, 6),)), Parse(word='памяти', tag=OpencorporaTag('NOUN,inan,femn 